### Imports

In [0]:
%tensorflow_version 1.x
import glob
import numpy as np
from itertools import chain
from music21 import converter, instrument, note, chord, stream
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, CuDNNLSTM, Dropout
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
import random

Using TensorFlow backend.


### Mounting google drive as a drive in the cloud

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True) 
dataset_path = "gdrive/My Drive/ML/music/Mozart/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


### Parsing Midi files:
Each file is parsed to notes and chords. If it is a chord, it is converted to a string so all contents. After this part, each file is converted to a list of consecutive notes and chords in string format and all lists are appended to the main list.

In [0]:
all_notes = []
for i, file in enumerate(glob.glob(dataset_path + "*.mid")):
  print(i, file)
  midi = converter.parse(file)
  parts = instrument.partitionByInstrument(midi)
  if parts: # file has instrument parts
      notes_to_parse = parts.parts[0].recurse()
  else: # file has notes in a flat structure
      notes_to_parse = midi.flat.notes

  notes = []
  for element in notes_to_parse:
      if isinstance(element, note.Note):
          notes.append(str(element.pitch))
      elif isinstance(element, chord.Chord):
          notes.append('.'.join(str(n) for n in element.normalOrder))
  all_notes.append(notes)
all_notes = [notes for notes in all_notes if len(notes) > 100] # Filter short files

0 gdrive/My Drive/ML/music/Mozart/mfig.mid
1 gdrive/My Drive/ML/music/Mozart/krebs.mid
2 gdrive/My Drive/ML/music/Mozart/magflute.mid
3 gdrive/My Drive/ML/music/Mozart/mozk211b.mid
4 gdrive/My Drive/ML/music/Mozart/mozk218b.mid
5 gdrive/My Drive/ML/music/Mozart/mozk281b.mid
6 gdrive/My Drive/ML/music/Mozart/mozk310c.mid
7 gdrive/My Drive/ML/music/Mozart/mozk309b.mid
8 gdrive/My Drive/ML/music/Mozart/mozk311b.mid
9 gdrive/My Drive/ML/music/Mozart/mozk331b.mid
10 gdrive/My Drive/ML/music/Mozart/mozk331c.mid
11 gdrive/My Drive/ML/music/Mozart/mozk332b.mid
12 gdrive/My Drive/ML/music/Mozart/mozk450b.mid
13 gdrive/My Drive/ML/music/Mozart/mozk488b.mid
14 gdrive/My Drive/ML/music/Mozart/mozk545b.mid
15 gdrive/My Drive/ML/music/Mozart/mozk545a.mid
16 gdrive/My Drive/ML/music/Mozart/mozk545c.mid
17 gdrive/My Drive/ML/music/Mozart/mozk622b.mid
18 gdrive/My Drive/ML/music/Mozart/mzflqnar.mid
19 gdrive/My Drive/ML/music/Mozart/mtmoz1.mid


Extracting distinct nodes and inferring the number of classes that the model needs to handle. Also creating a mapping from each string to a numerical value.


In [0]:
notes_flattened = list(chain.from_iterable(all_notes))
pitch_names = sorted(set(notes_flattened))
print(len(pitch_names))
note_to_int = dict((note, number) for number, note in enumerate(pitch_names))

200


For every file, a window of size sequence_length scans the notes of that file and chooses that window as input and the following note as output.

In [0]:
seq_length = 50
X = []
y = []
for notes in all_notes:
  mapped_notes = [note_to_int[n] for n in notes]
  for i in range(len(notes) - seq_length):
    X.append(mapped_notes[i:i+seq_length])
    y.append(mapped_notes[i+seq_length])

In [0]:
uniqueValues, occurCount = np.unique(y, return_counts=True)
 
print("Unique Values : " , uniqueValues)
print("Occurrence Count : ", occurCount)

Unique Values :  [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199]
Occurrence Count :  [  45    1   92    1    3    1   33    3   18    4  201   15   31    7
   32    3   72    5    3    3    6    7   33   43    1    2    5    1
    1   

Lists are siwtched to numpy arrays and outputs are switched to one_hot encoding so that the model is able to handle them.

In [0]:
X = np.array(X)
X = X.reshape(X.shape[0], X.shape[1], 1)
X = X / len(pitch_names) # normalize input
y = to_categorical(y)
print(X.shape)

(23933, 50, 1)


### Model Architecture:
2 layers of LSTM node, the first one is of size 128 or 256 nodes and the second one is of size 256 or 512 nodes. Dropout regularization technique is used to prevent nodes from depending on each others. 1 hidden layer of size 256 is appended to the model and finally the classification layer.

In [0]:
def LSTM_model(X, n_notes):
  model = Sequential()
  model.add(CuDNNLSTM(256, input_shape=(X.shape[1], 1), return_sequences=True))
  model.add(Dropout(0.2))
  model.add(CuDNNLSTM(512))
  model.add(Dropout(0.2))
  # model.add(CuDNNLSTM(128))
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(n_notes, activation='softmax'))
  return model

The model uses categorical crossentropy as a loss function and adam optimizer. We also checkpoint the best weights during training so that they are loaded the next time the model is trained and not start from scratch.
Batch size is 128 because this makes the model complete one epoch more faster.

In [0]:
model = LSTM_model(X, len(pitch_names))
# model.load_weights('gdrive/My Drive/ML/LSTM_music_weights_ReducedMozart_seq50_256_512_512.hdf5')
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
checkpointer = ModelCheckpoint(
    filepath='gdrive/My Drive/ML/LSTM_music_weights_ReducedMozart_seq50_256_512_512.hdf5',
    monitor='loss', 
    verbose=1, 
    save_best_only=True) # To Checkpoint weights
model.fit(X, y, callbacks=[checkpointer], epochs=100, batch_size=256)
model.save('gdrive/My Drive/ML/LSTM_mozart_model_ReducedMozart_seq50_256_512_512.h5')

Epoch 1/100
23933/23933 [==============================] - 12s 500us/step - loss: 4.3294 - acc: 0.0359

Epoch 00001: loss improved from inf to 4.32940, saving model to gdrive/My Drive/ML/LSTM_music_weights_ReducedMozart_seq50_256_512_512.hdf5
Epoch 2/100
23933/23933 [==============================] - 11s 449us/step - loss: 4.1752 - acc: 0.0425

Epoch 00002: loss improved from 4.32940 to 4.17521, saving model to gdrive/My Drive/ML/LSTM_music_weights_ReducedMozart_seq50_256_512_512.hdf5
Epoch 3/100
23933/23933 [==============================] - 11s 450us/step - loss: 4.1131 - acc: 0.0473

Epoch 00003: loss improved from 4.17521 to 4.11306, saving model to gdrive/My Drive/ML/LSTM_music_weights_ReducedMozart_seq50_256_512_512.hdf5
Epoch 4/100
23933/23933 [==============================] - 11s 453us/step - loss: 4.0691 - acc: 0.0520

Epoch 00004: loss improved from 4.11306 to 4.06913, saving model to gdrive/My Drive/ML/LSTM_music_weights_ReducedMozart_seq50_256_512_512.hdf5
Epoch 5/100
2393

### Music generation:
Music is generated by selecting a random sample from the training set and pass it to the model to predict. the outcome of the model is appended to the input sequnce and now the new input to the model is only the last 20 notes of that sample and so on until 50 or more new notes are generated. The outputs are switched back to string format.
Sometimes, the model tend to be biased towards one note, that's why we choose randomly between the highest 2 probabilities generated by the model to avoid this problem.

In [0]:
start = np.random.randint(0, X.shape[0], 1)
int_to_note = dict((number, note) for number, note in enumerate(pitch_names))
pattern = X[start]
prediction_output = []
for i in range(50):
  start_input = pattern
  output = model.predict(pattern, verbose=0)
  # print(output)
  # possible_outcomes = (-output).argsort()
  # index1 = possible_outcomes[0][0]
  # index2 = possible_outcomes[0][1]
  # if random.choice((True, False)):
  #   index = index1
  # else:
  #   index = index2
  index = np.argmax(output)
  result = int_to_note[index]
  prediction_output.append(result)
  pattern = pattern.reshape(pattern.shape[1])
  pattern = np.append(pattern, index)
  pattern = pattern[1:pattern.shape[0]]
  pattern = pattern.reshape(1, pattern.shape[0], 1)  

Music21 library is used to generate music. Piano insturment takes as input the chord and produces the actual sound. Finally, the stream of notes is converted to a Midi file.

In [0]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    offset += 1

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='gdrive/My Drive/ML/music/Mozart_top1notes_offset10.mid')

'gdrive/My Drive/ML/music/Mozart_top1notes_offset10.mid'